# Basic Fitting Workflow

This notebook demonstrates the standard fitting workflow for time- and energy-resolved spectroscopy data.

**Example System:**
- Single Gaussian-Lorentzian Product (GLP) peak
- Components: offset + linear background
- Time dependence: exponential shift of peak position (with IRF convolution)

**Workflow:**
1. Load and inspect data
2. Fit baseline (ground state) spectrum
3. Slice-by-Slice fitting to identify time-dependent parameters
4. Global 2D fitting with parametric time dependence

In [ ]:
import os
import numpy as np
import trspecfit
from trspecfit.utils.lmfit import MC  # For uncertainty estimation

## 1. Load Data

Load simulated time- and energy-resolved spectroscopy data from CSV files.

In [ ]:
# Create project
project = trspecfit.Project(path=os.getcwd())

In [ ]:
# Load data from CSV files
data_folder = "data"

file = trspecfit.File(
    parent_project=project,
    path=data_folder,
    data=np.loadtxt(project.path / data_folder / "data.csv", delimiter=','),
    energy=np.loadtxt(project.path / data_folder / "energy.csv"),
    time=np.loadtxt(project.path / data_folder / "time.csv")
)

print(f"Data shape: {file.data.shape}")
print(f"Energy range: {file.energy.min():.1f} - {file.energy.max():.1f}")
print(f"Time range: {file.time.min():.1f} - {file.time.max():.1f}")

## 2. Inspect Data

Visualize the raw data to identify features and appropriate fitting regions.

In [ ]:
# Visualize full dataset
file.describe()

## 3. Define Fitting Region

Set energy and time limits to focus on the region of interest.

In [ ]:
# Set fitting limits (absolute values)
file.set_fit_limits(
    energy_limits=[5, 18],  # Energy range
    time_limits=[-10, 99]  # Time range
)

## 4. Define Baseline Spectrum

Extract the ground state/pre-trigger spectrum by averaging over early time points.

In [ ]:
# Define baseline using time indices
file.define_baseline(
    time_start=0,
    time_stop=12,
    time_type='ind'  # Use indices (or 'abs' for absolute time values)
)

## 5. Fit Baseline Spectrum

Fit the ground state spectrum to establish initial parameters for time-dependent fits.

In [ ]:
# Load baseline model
file.load_model(
    model_yaml="models_energy.yaml",
    model_info=["base"]
)

# Inspect model structure and parameters
file.describe_model(model_info="base", detail=0)

In [ ]:
# Fit baseline
# fit=0: show initial guess only
# fit=1: single optimization
# fit=2: two-stage (global + local)
file.fit_baseline(model_name="base", fit=2)

## 6. (Optional) Slice-by-Slice Fitting

Fit each time slice independently to get an idea of how parameters evolve over time.

In [ ]:
# Load Slice-by-Slice model
file.load_model(
    model_yaml="models_energy.yaml",
    model_info=["SbS"]
)

file.describe_model("SbS", detail=0)

In [ ]:
# Perform Slice-by-Slice fit
file.fit_SliceBySlice(
    model_name="SbS",
    fit=1,
    try_CI=0  # Set to 1 to calculate confidence intervals (slower)
)

## 7. Global 2D Fitting

Fit the entire 2D dataset simultaneously with time-dependent parameters.

### Key Advantage:
Global fitting constrains parameters across all time points, reducing overfitting and providing more robust parameter estimates.

In [ ]:
# Load 2D model
file.load_model(
    model_yaml="models_energy.yaml",
    model_info=["2D"]
)

file.describe_model(model_info="2D", detail=0)

In [ ]:
# Add time dependence to parameter
file.add_time_dependence(
    model_yaml="models_time.yaml",
    model_info=["MonoExpPosIRF"],
    par_name="GLP_01_x0"  # Peak position varies with time
)

print("\n=== Model with Time Dependence ===")
file.describe_model(model_info=["2D"], detail=1)

In [ ]:
# Configure uncertainty estimation (optional)
mc_settings = MC(
    useMC=0,  # Set to 1 to use Monte Carlo uncertainty estimation
    steps=5000,
    nwalkers=20,
    thin=1
)

# Perform global 2D fit
file.fit_2Dmodel(
    model_name="2D",
    fit=2,
    try_CI=0,  # Set to 1 for confidence intervals
    MCsettings=mc_settings
)

## Tips for Basic Fitting

**Fitting Strategy:**
- Always fit baseline first to establish good initial values
- Use Slice-by-Slice to identify which parameters are time-dependent
- Apply time-dependence only to parameters that clearly evolve
- Start with simple time-dependencies (exponential) before complex ones

**Fit Levels:**
- `fit=0`: Show initial guess only (useful for checking model setup)
- `fit=1`: Single optimization pass
- `fit=2`: Two-stage fitting (global + local) for more robust results

**Uncertainty Estimation:**
- `try_CI=1`: Fast linear approximation of confidence intervals
- `MC` with `useMC=1`: More robust Monte Carlo estimation

**Next Steps:**
- See `02_dependent_parameters` for fitting with parameter constraints
- See `03_multi_cycle` for fitting datasets with multiple sub-cycles